Imports


In [8]:


import torch


from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
import pandas as pd
import mne
import numpy as np
from scipy.fftpack import fft, ifft
from sklearn.preprocessing import LabelEncoder
#NOTE USE LABEL ENCODER BEFORE RUNNING THE FULL VERSION 

Dataset and preprocessing


In [ ]:
#this is where we actually extract the features 
def get_individual_frequency_power_bands(surrogate_segment)
    
    power_features=[]
    
    
    #Create band-pass filters for each frequency band of interest.
    delta_band = raw.copy().filter(l_freq=0.5, h_freq=4, picks='eeg')
    theta_band = raw.copy().filter(l_freq=4, h_freq=8, picks='eeg')
    alpha_band = raw.copy().filter(l_freq=8, h_freq=13, picks='eeg')
    beta_band = raw.copy().filter(l_freq=13, h_freq=30, picks='eeg')
    gamma_band = raw.copy().filter(l_freq=30, h_freq=None, picks='eeg')  # Assuming Gamma is 30+ Hz
    
    #Use the psd_welch method to calculate the PSD for each filtered band.
    delta_psd, delta_freq = mne.time_frequency.psd_welch(delta_band)
    theta_psd, theta_freq = mne.time_frequency.psd_welch(theta_band)
    alpha_psd, alpha_freq = mne.time_frequency.psd_welch(alpha_band)
    beta_psd, beta_freq = mne.time_frequency.psd_welch(beta_band)
    gamma_psd, gamma_freq = mne.time_frequency.psd_welch(gamma_band)

    #Integrate the PSD over the frequencies of interest to get the absolute power for each band.
    delta_power = delta_psd.mean(axis=0).sum()  
    theta_power = theta_psd.mean(axis=0).sum()  
    alpha_power = alpha_psd.mean(axis=0).sum()  
    beta_power = beta_psd.mean(axis=0).sum()    
    gamma_power = gamma_psd.mean(axis=0).sum()  
    
    power_features.extend(delta_power,theta_power,alpha_power,beta_power,gamma_power)
    
def ratios_for_power_bands(delta_power,theta_power,alpha_power,beta_power,gamma_power)

In [43]:
def extract_features(surrogate_segment):
    features=[]
    eeg_data=np.real(surrogate_segment)
    
    
    features.append(get_individual_frequency_power_bands(eeg_data))

def create_raw_from_parquet(parquet_file):
    df=pd.read_parquet(parquet_file)
    data=df.to_numpy().T
    info=mne.create_info(ch_names=list(df.columns),sfreq=200,ch_types='eeg')
    raw=mne.io.RawArray(data,info)
    return raw

def get_duration(raw):
    num_samples=len(raw.times)
    sampling_freq=raw.info['sfreq']
    duration=np.floor(num_samples/sampling_freq)
    
    return duration

In [51]:



# Initialize an empty DataFrame for the new training data
columns = [
    'eeg_id', 
    'eeg_sub_id', 
    'patient_id', 
    'Standard Deviation (STD)', 
    'Inter-Quartile Range (IQR)', 
    'Skewness', 
    'Kurtosis', 
    'Number of Zero Crossings', 
    'Hjorth Mobility', 
    'Hjorth Complexity', 
    'Higuchi Fractal Dimension', 
    'Shannon Entropy', 
    'Spectral Entropy', 
    'Binned Entropy', 
    'Delta Power', 
    'Theta Power', 
    'Alpha Power', 
    'Beta Power', 
    'Gamma Power', 
    'Delta/Theta Ratio', 
    'Delta/Alpha Ratio', 
    'Theta/Beta Ratio', 
    'Alpha/Gamma Ratio', 
    'Beta/Gamma Ratio', 
    'Delta+Theta Power', 
    'Alpha+Beta Power', 
    'Theta+Gamma Power', 
    'Total Power',
    'expert_consensus'
]

new_train_df = pd.DataFrame(columns=columns)

# Load the original training CSV
train_csv = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
#count=0
# Loop through each row in the train CSV
for index, row in train_csv.iterrows():
    eeg_id = row['eeg_id']
    sub_id = row['eeg_sub_id']
    patient_id = row['patient_id']
    seizure_label = row['expert_consensus']
   
    # Load the corresponding parquet file as an mne Raw object
    raw = create_raw_from_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    
    sub_id_start_time=sub_id
    tmin = sub_id_start_time * 50
    tmax = tmin + 50  # This ensures a 50-second window

    # Adjust tmax to not exceed the recording
    tmax = min(tmax, raw.times[-1])
    
    # Additionally, ensure tmin does not exceed the adjusted tmax or the recording
    tmin = min(tmin, tmax - 0.001)
    segment = raw.crop(tmin, tmax)  # Adjust 'tmin' and 'tmax' as necessary
    
    # Generate FT Surrogate for the segment
    ft_segment = fft(segment.get_data())
    random_phases = np.exp(2j * np.pi * np.random.rand(*ft_segment.shape))
    surrogate_data = np.abs(ft_segment) * random_phases
    surrogate_segment = ifft(surrogate_data)
    
    # Extract features from the surrogate segment
    features = extract_features(surrogate_segment)
    
    # Append to the new DataFrame
    new_row = [eeg_id, sub_id, patient_id] + features + [seizure_label]
    new_train_df.loc[len(new_train_df)] = new_row
    '''''count+=1
    print(count)
    if count==10:
        break''''
# Save the new DataFrame to CSV
#new_train_df.to_csv('path/to/new_train_data.csv', index=False)


1
2
3
4
5
6
7
8
9
10


In [52]:
print(new_train_df)

       eeg_id  eeg_sub_id  patient_id  Standard Deviation(STD)  \
0  1628180742           0       42516                      1.5   
1  1628180742           1       42516                      1.5   
2  1628180742           2       42516                      1.5   
3  1628180742           3       42516                      1.5   
4  1628180742           4       42516                      1.5   
5  1628180742           5       42516                      1.5   
6  1628180742           6       42516                      1.5   
7  1628180742           7       42516                      1.5   
8  1628180742           8       42516                      1.5   
9  2277392603           0       30539                      1.5   

   Inter-Quartile Range (IQR)  Skewness  Kurtosis  Number of Zero Crossings  \
0                         2.0       1.5       2.0                       1.5   
1                         2.0       1.5       2.0                       1.5   
2                         2.0       

In [6]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')

# Calculate the percentage of each label
label_percentage = df['expert_consensus'].value_counts(normalize=True) * 100

print(label_percentage)


expert_consensus
Seizure    19.600187
GRDA       17.660112
Other      17.610487
GPD        15.638577
LRDA       15.580524
LPD        13.910112
Name: proportion, dtype: float64


Model Architure
